<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
kr_dst run notebook</p><br>

# Runs the kr_dst

In [1]:
import os
import sys
import glob
import logging
import math
import numpy  as np
import warnings
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from   dataclasses import dataclass

In [2]:
from nextflex.core import Setup
from nextflex.core import PosQ

In [3]:
from nextflex.core import get_evt_true_positions_and_energy
from nextflex.core import get_evt_true_positions_df
from nextflex.core import get_sensor_response
from nextflex.core import sensor_response_ti
from nextflex.core import event_sensor_response_ti
from nextflex.core import sensor_number_response_ti

In [4]:
from nextflex.core import mcparts_and_sensors_response
from nextflex.core import get_s1
from nextflex.core import get_s2
from nextflex.core import get_qtot

In [5]:
from nextflex.core import get_q
from nextflex.core import get_pos
from nextflex.core import get_position
from nextflex.core import diff_pos

In [20]:
from nextflex.krypton_dst import kr_dst
from nextflex.krypton_dst import kr_join_dst

In [7]:
from nextflex.krypton_analysis import histo_df
from nextflex.krypton_analysis import histos_df
from nextflex.krypton_analysis import histo1d
from nextflex.krypton_analysis import histos1d

In [8]:
import invisible_cities.core.system_of_units  as units
from invisible_cities.core.core_functions     import in_range

from invisible_cities.io.mcinfo_io import get_sensor_binning
from invisible_cities.io.mcinfo_io import load_mcparticles_df
from invisible_cities.io.mcinfo_io import load_mcsensor_response_df

In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Run configuration

In [10]:
FDATA = os.environ['FLEXDATA']
print(f'path to data ={FDATA}')

path to data =/Users/jj/Development/flexdata


### Setup

***
@dataclass\
class Setup:

    sipmPDE           : float = 1.0
    maskPDE           : float = 1.0
    qTh               : float = 0.0
    tpConfig          : str   = "FLEX100_M6_O6" # means thickness 6 mm hole 6mm
    mapDIR            : str   = "flexmaps" # where to find the SiPM map
    fibres            : bool  = False

***


In [11]:
setup = Setup(sipmPDE    = 1.0,
              maskPDE    = 1.0,
              qTh        = 0.0,
              tpConfig   = "FLEX100_M6_O6", 
              mapDIR     = "flexmaps",
              fibres     = False)
print(setup)


        Setup <PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0_FLEX100_M6_O6>:
        tracking plane configuration = FLEX100_M6_O6
        sipm PDE                     = 1.0
        transmission of teflon masks = 1.0
        charge threshold             = 0.0
        Fibres?                      = False
        


### SiPM map

In [12]:
mapFile       = os.path.join(FDATA,setup.mapDIR, 'sipm_map.csv')
sipm_map      = pd.read_csv(mapFile)
sipm_map.head()

,Unnamed: 0,sensor_id,sensor_name,x,y,xl,xr,yu,yd,id_xl,id_xr,id_yu,id_yd
0,0,1040,TP_SiPM,15.55,-450.95,1.907349e-07,31.100000,-435.400012,-466.500012,1039,1041,1066,1019
1,1,4059,TP_SiPM,93.30,450.95,7.775000e+01,108.850003,466.500012,435.400012,4058,4060,4080,4033
2,2,3753,TP_SiPM,-62.20,326.55,-7.775000e+01,-46.650001,342.099988,310.999988,3752,3754,3799,3705
3,3,3842,TP_SiPM,-77.75,357.65,-9.330000e+01,-62.200000,373.199994,342.099994,3841,3843,3884,3798
4,4,3797,TP_SiPM,-93.30,342.10,-1.088500e+02,-77.750003,357.650006,326.550006,3796,3798,3841,3751


### Directory for files describing the SiPM plane configuration

In [14]:
ifnames = glob.glob(f"{FDATA}/{setup.tpConfig}/*.h5")
print(f'found {len(ifnames)} files')

found 100 files


### Extract kr_dsts from h5

In [20]:
gf, bf = kr_dst(ifnames, sipm_map, setup, ic=20)
print(f'good files ={len(gf)}')
print(f'bad files ={len(bf)}')


reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1023.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1023.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1032.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1032.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1031.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1031.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1024.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1024.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1018.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX10

### Join kr_dsts 

In [15]:
ifnames = glob.glob(f"{FDATA}/{setup.tpConfig}/*.csv")
print(f'found {len(ifnames)} files')

found 100 files


In [16]:
ofile_name = f"{setup.name}.csv"
print(f"output file name = {ofile_name}")

output file name = PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0_FLEX100_M6_O6.csv


In [22]:
ofile = f"{FDATA}/kdsts/{ofile_name}"
print(f"Path to ofile = {ofile}")

Path to ofile = /Users/jj/Development/flexdata/kdsts/PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0_FLEX100_M6_O6.csv


In [23]:
krdf, BF = kr_join_dst(ifnames, verbose=False)
print(f'bad files ={len(BF)}')
print(krdf)
krdf.to_csv(ofile)

bad files =0
    event_id      true_x      true_y      true_z         KE  S1e   S2e  Qtot  \
0     101900  359.476500    9.660492   722.39014  55.389090   18  8395   794   
1     101901  186.837890  -43.123417  1162.73030  41.543297   22  8774   747   
2     101902 -259.909730  -53.806812  1087.68680  54.014584   29  8693   829   
3     101903 -399.442720 -187.637620   699.62256  41.543297   28  7923   756   
4     101904   77.765274  -40.491470   991.10700  43.017110   35  8940   771   
..       ...         ...         ...         ...        ...  ...   ...   ...   
95    103595 -183.870450  435.716060   677.17190  41.543297   24  6810   614   
96    103596  273.419280 -302.450700  1186.15840  54.050537   20  8160   780   
97    103597  267.384200 -260.087040   634.82007  41.543297   31  8315   784   
98    103598  406.801400 -110.261090   435.15662  41.543297   11  7737   760   
99    103599  135.363780  -17.823605   641.65350  41.543297   19  8711   830   

      xMax    yMax  ...   

## FLEX100_M6_O6 with SiPM = 0.4

In [38]:
ifnames = glob.glob(f"{FDATA}/{setup.tpConfig}/*.h5")
print(f'found {len(ifnames)} files')

found 100 files


In [39]:
setup = Setup(sipmPDE    = 0.4,
              maskPDE    = 1.0,
              qTh        = 0.0,
              tpConfig   = "FLEX100_M6_O6")
print(setup)


        Setup <PMTs_sipmPDE_0.4_maskPDE_1.0_qTh_0.0_FLEX100_M6_O6>:
        tracking plane configuration = FLEX100_M6_O6
        sipm PDE                     = 0.4
        transmission of teflon masks = 1.0
        charge threshold             = 0.0
        Fibres?                      = False
        


In [40]:
gf, bf = kr_dst(ifnames, sipm_map, setup, ic=20)
print(f'good files ={len(gf)}')
print(f'bad files ={len(bf)}')


reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1023.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1023.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1032.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1032.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1031.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1031.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1024.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1024.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1018.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX10

In [41]:
ifnames2 = glob.glob(f"{FDATA}/{setup.tpConfig}/*.csv")
print(f'found {len(ifnames2)} files')

found 100 files


In [42]:
ofile_name = f"{setup.name}.csv"
ofile = f"{FDATA}/kdsts/{ofile_name}"
print(f"Path to ofile = {ofile}")

Path to ofile = /Users/jj/Development/flexdata/kdsts/PMTs_sipmPDE_0.4_maskPDE_1.0_qTh_0.0_FLEX100_M6_O6.csv


In [43]:
krdf, BF = kr_join_dst(ifnames2, verbose=False)
print(f'bad files ={len(BF)}')
#print(krdf)
krdf.to_csv(ofile)

bad files =0


## FLEX100_M6_O6 with SiPM = 0.4, qth > 5

In [44]:
ifnames = glob.glob(f"{FDATA}/{setup.tpConfig}/*.h5")
print(f'found {len(ifnames)} files')

found 100 files


In [45]:
setup = Setup(sipmPDE    = 0.4,
              maskPDE    = 1.0,
              qTh        = 5.0,
              tpConfig   = "FLEX100_M6_O6")
print(setup)


        Setup <PMTs_sipmPDE_0.4_maskPDE_1.0_qTh_5.0_FLEX100_M6_O6>:
        tracking plane configuration = FLEX100_M6_O6
        sipm PDE                     = 0.4
        transmission of teflon masks = 1.0
        charge threshold             = 5.0
        Fibres?                      = False
        


In [46]:
gf, bf = kr_dst(ifnames, sipm_map, setup, ic=50)
print(f'good files ={len(gf)}')
print(f'bad files ={len(bf)}')

reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1023.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1023.next.csv, with 100 events
reading file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6.Kr83.ACTIVE.1091.next.h5
saving file /Users/jj/Development/flexdata/FLEX100_M6_O6/FLEX100_M6_O6Kr83.ACTIVE.1091.next.csv, with 100 events
good files =100
bad files =0


In [47]:
ofile_name = f"{setup.name}.csv"
ofile = f"{FDATA}/kdsts/{ofile_name}"
print(f"Path to ofile = {ofile}")

Path to ofile = /Users/jj/Development/flexdata/kdsts/PMTs_sipmPDE_0.4_maskPDE_1.0_qTh_5.0_FLEX100_M6_O6.csv


In [48]:
krdf, BF = kr_join_dst(ifnames2, verbose=False)
print(f'bad files ={len(BF)}')
#print(krdf)
krdf.to_csv(ofile)

bad files =0
